In [1]:
from tda import auth, client
import json

token_path = R'X:\GitHub\beatingENd\BeatingEnDUS\Account Tracking\tdaAPI2\token.pickle'
api_key = 'DRSMU4TL964FO3QNBQHVL78X9SUPGGIL@AMER.OAUTHAP'
redirect_uri = "http://localhost"
try:
    c = auth.client_from_token_file(token_path, api_key)
except FileNotFoundError:
    from selenium import webdriver
    with webdriver.Chrome() as driver:
        c = auth.client_from_login_flow(
            driver, api_key, redirect_uri, token_path)

In [12]:
p = client.Client.Account.Fields("positions")

In [13]:
a = c.get_accounts(fields=p)

In [14]:
assert a.status_code == 200, a.raise_for_status()
data = a.json()

In [3]:
import pandas as pd

In [15]:
dictd = data[0]

In [16]:
df = pd.json_normalize(data[0])

In [17]:
account_dataframes = []
for accountId, value in dictd.items():
    account_dataframes.append(pd.io.json.json_normalize(value))
    account_dataframes[-1].columns = [
        c.replace("securitiesAccount.", "")
        for c in account_dataframes[-1].columns
    ]

pd.concat(account_dataframes)

type  accountId  roundTrips  isDayTrader  isClosingOnlyRestricted  \
0  MARGIN  220535376           0        False                    False   

                                           positions  \
0  [{'shortQuantity': 0.0, 'averagePrice': 42.43,...   

   initialBalances.accruedInterest  \
0                              0.0   

   initialBalances.availableFundsNonMarginableTrade  \
0                                           2460.56   

   initialBalances.bondValue  initialBalances.buyingPower  ...  \
0                        0.0                      5789.56  ...   

   currentBalances.bondValue  projectedBalances.availableFunds  \
0                        0.0                           2445.16   

   projectedBalances.availableFundsNonMarginableTrade  \
0                                            2445.16    

   projectedBalances.buyingPower  projectedBalances.dayTradingBuyingPower  \
0                        5789.56                                      0.0   

   projectedBalances.dayTradingBuyingPowerCall  \
0                                          0.0   

   projectedBalances.maintenanceCall  projectedBalances.regTCall  \
0                                0.0                         0.0   

   projectedBalances.isInCall  projectedBalances.stockBuyingPower  
0                       False                             5789.56  

[1 rows x 75 columns]

In [18]:
raw_positions = pd.json_normalize(df["securitiesAccount.positions"][0])

In [19]:
raw_positions[["currentDayProfitLossPercentage", "instrument.symbol"]]

currentDayProfitLossPercentage  instrument.symbol
0                            -0.74               XPEV
1                             0.00  AAPL_031921P127.5
2                             0.00    AAPL_031921P130
3                             0.00    PTON_031921P120
4                             0.00    PTON_031921P125
5                             0.00     DIS_031921P170
6                             0.00     DIS_031921P175
7                            -1.26                NIO
8                             0.00     AMD_031921C100
9                             0.00      AMD_012122C80
10                            0.00      RKT_031921P16
11                            0.00      NET_031921P75
12                            0.00      NET_031921P80
13                            0.00      FB_031921P250
14                            0.00      FB_031921P255

## Option Testing

In [3]:
para = c.Options

In [4]:
chains = c.get_option_chain(symbol="AAPL", contract_type=para.ContractType.PUT, strategy=para.Strategy.VERTICAL, option_type=para.Type.STANDARD)

In [6]:
initial = pd.json_normalize(chains.json(), record_path="monthlyStrategyList")
filtered = initial[initial["daysToExp"] < 42]

In [7]:
filtered

month  year  day  daysToExp secondaryMonth  secondaryYear  secondaryDay  \
0   Feb  2021   26          5            Feb           2021            26   
1   Mar  2021    5         12            Mar           2021             5   
2   Mar  2021   12         19            Mar           2021            12   
3   Mar  2021   19         26            Mar           2021            19   
4   Mar  2021   26         33            Mar           2021            26   
5   Apr  2021    1         39            Apr           2021             1   

   secondaryDaysToExp type secondaryType   leap  \
0                   5    P             P  False   
1                  12    P             P  False   
2                  19    P             P  False   
3                  26    P             P  False   
4                  33    P             P  False   
5                  39    P             P  False   

                                  optionStrategyList  secondaryLeap  
0  [{'primaryLeg': {'symbol': 'AAPL_022621P101', ...          False  
1  [{'primaryLeg': {'symbol': 'AAPL_030521P119', ...          False  
2  [{'primaryLeg': {'symbol': 'AAPL_031221P119', ...          False  
3  [{'primaryLeg': {'symbol': 'AAPL_031921P116', ...          False  
4  [{'primaryLeg': {'symbol': 'AAPL_032621P119', ...          False  
5  [{'primaryLeg': {'symbol': 'AAPL_040121P119', ...          False

In [9]:
spreads = pd.json_normalize(filtered["optionStrategyList"].explode())

In [11]:
rando_data

strategyStrike  strategyBid  strategyAsk primaryLeg.symbol  \
6       65.0/60.0         1.95         2.35    FTCH_031921P65   
33      60.0/55.0         1.65         2.20     RUN_031921P60   
9     115.0/110.0         1.90         2.60      U_031921P115   

   primaryLeg.putCallInd   primaryLeg.description  primaryLeg.bid  \
6                      P  FTCH Mar 19 2021 65 Put            5.55   
33                     P   RUN Mar 19 2021 60 Put            4.20   
9                      P    U Mar 19 2021 115 Put            6.80   

    primaryLeg.ask primaryLeg.range  primaryLeg.strikePrice  \
6             5.75              OTM                    65.0   
33            4.50              OTM                    60.0   
9             7.10              OTM                   115.0   

    primaryLeg.totalVolume secondaryLeg.symbol secondaryLeg.putCallInd  \
6                    173.0      FTCH_031921P60                       P   
33                  1260.0       RUN_031921P55                       P   
9                    396.0        U_031921P110                       P   

   secondaryLeg.description  secondaryLeg.bid  secondaryLeg.ask  \
6   FTCH Mar 19 2021 60 Put               3.4              3.60   
33   RUN Mar 19 2021 55 Put               2.3              2.55   
9     U Mar 19 2021 110 Put               4.5              4.90   

   secondaryLeg.range  secondaryLeg.strikePrice  secondaryLeg.totalVolume  
6                 OTM                      60.0                     222.0  
33                OTM                      55.0                     232.0  
9                 OTM                     110.0                     709.0

In [27]:
rando_data = []

for i in df["Ticker"].to_list():
    try: 
        chains = c.get_option_chain(symbol=i, contract_type=para.ContractType.PUT, strategy=para.Strategy.VERTICAL, option_type=para.Type.STANDARD)
        initial = pd.json_normalize(chains.json(), record_path="monthlyStrategyList")
        filtered = initial[initial["daysToExp"] < 45]
        spreads = pd.json_normalize(filtered["optionStrategyList"].explode())
        output = spreads.loc[(spreads["strategyBid"]>1.40) & (spreads["primaryLeg.range"]=="OTM")]
        rando_data.append(output)

    except:
        pass

rando_data = pd.concat(rando_data)

In [34]:
possible = rando_data.to_html().replace("\n", "")

# Email Testing

In [18]:
import yagmail

In [19]:
yag = yagmail.SMTP("trashmail.imtrash@gmail.com","trash6969")

In [35]:
name = ["Your positions: \n\n", "Finviz scan: \n\n", recco, "Possible new positions: \n\n", possible]

In [36]:
yag.send(to='zacharylim98@gmail.com', contents=name)

{}

In [72]:
position = raw_positions[["instrument.symbol", "averagePrice"]].to_html().replace("\n","")

# Finviz Scanning

In [4]:
from finviz.screener import Screener
import nest_asyncio
nest_asyncio.apply()

In [5]:
filter = ["an_recom_buybetter","cap_midover","sh_avgvol_o2000","sh_curvol_o5000","sh_opt_option","sh_price_o10","ta_pattern_horizontal","ta_perf_ddown"]
daily_scan = Screener(filters=filter, table="Overview")

In [6]:
df = pd.json_normalize(daily_scan.data)

In [29]:
dfi.export(df.iloc[:20,:], "df1.jpeg")
dfi.export(df.iloc[20:40,:], "df2.jpeg")
dfi.export(df.iloc[40:60,:], "df3.jpeg")
dfi.export(df.iloc[60:80,:], "df4.jpeg")
dfi.export(df.iloc[80:100,:], "df5.jpeg")

In [55]:
file_object = [open(".\\" + "df" + str(i) + ".jpeg", "rb") for i in range(1,5)]

In [56]:
file_object

[<_io.BufferedReader name='.\\df1.jpeg'>,
 <_io.BufferedReader name='.\\df2.jpeg'>,
 <_io.BufferedReader name='.\\df3.jpeg'>,
 <_io.BufferedReader name='.\\df4.jpeg'>]

In [8]:
df["Chart_URL"] = ["https://finviz.com/chart.ashx?ty=c&p=d&b=1&ta=1&t={}".format(i) for i in df["Ticker"]]

In [9]:
recco = df.to_html(render_links=True).replace("\n","")

# Bible Verse Testing

In [61]:
from bs4 import BeautifulSoup
import requests

def verse_otd():    
    URL = "https://www.bible.com/en-GB"
    page = requests.get(URL)

    soup = BeautifulSoup(page.content, 'html.parser')
    verse = soup.find('p', class_="votd-verse").text.strip()
    ref = soup.find('p', class_="votd-ref").text.strip()
    return(verse, ref)

In [62]:
verse_otd()

('The heavens declare the glory of God; the skies proclaim the work of his hands.',
 'Psalms 19:1 (NIV)')

In [191]:
def spec_verse(book="EPH", chp="3", verse="20"):

    books = {"Genesis" : "GEN", "Exodus" : "EXO"}

    book_norm = dict_bible[book]

    URL = "https://www.bible.com/en-GB/bible/59/{}.{}.{}".format(book_norm, chp, verse)
    page = requests.get(URL)

    soup= BeautifulSoup(page.content, 'html.parser')
    verse = soup.find('div', class_="near-black lh-copy f3-m").text.strip()
    return(verse)

In [197]:
spec_verse(book="Ephesians", chp="3", verse="20")

'Now to him who is able to do far more abundantly than all that we ask or think, according to the power at work within us'

In [164]:
books = {"Genesis" : "GEN", "Exodus" : "EXO"}

In [169]:
cross_ref = pd.read_csv(R"X:\GitHub\personal-improvement\books of bible\crossref.csv")

In [185]:
crd = dict(cross_ref.values)

In [188]:
dict_bible = dict([(value, key) for key, value in crd.items()]) 

In [190]:
dict_bible["Genesis"]

'Gen'

# Telegram Sending

In [7]:
import telegram_send

In [63]:
telegram_send.configure(R"X:\GitHub\beatingENd\BeatingEnDUS\Account Tracking\tdaAPI2\telegramtoken.pickle")

Talk with the BotFather on Telegram (https://telegram.me/BotFather), create a bot and insert the token
Connected with dailysend_bot.

Please add dailysend_bot on Telegram (https://telegram.me/dailysend_bot)
and send it the password: 64686

Congratulations Krismerful! 
telegram-send is now ready for use!


In [60]:
telegram_send.send(messages=["Hi"], images=[file_object1, file_object2, file_object3, file_object4], conf=R".\telegramtoken.pickle",parse_mode="html")

In [8]:
import dataframe_image as dfi

In [59]:
# dfi.export(df,"mytable.jpeg")
file_object1  = open(R".\df1.jpeg", "rb") 
file_object2  = open(R".\df2.jpeg", "rb") 
file_object3  = open(R".\df3.jpeg", "rb") 
file_object4  = open(R".\df4.jpeg", "rb") 

In [81]:
telegram_send.send(messages=["Good Morning Zach! Here's the verse for the day:", verse,  "Here's a reminder that you are loved and you don't have to take things personally to prove yourself"], files=[ticked], conf=R".\telegramtoken.pickle",parse_mode="markdown")

In [74]:
tikerlist = df["Ticker"].to_list()

In [79]:
tikerlist

f=open('DailyScan.txt','w')
for ele in tikerlist:
    f.write(ele+',')

f.close()

In [80]:
ticked = open(R".\DailyScan.txt", 'rb')

In [93]:
dfi.export(df.sort_values(["Sector","Change"], ascending=False).drop(["Country", "Industry", "Volume"], axis=1), "finviz.jpeg")